In [19]:
import tensorflow as tf
import numpy as py
import matplotlib.pyplot as plt
import input_data

In [20]:
# 加载数据集
mnist = input_data.read_data_sets('data/',one_hot=True)
trainimgs = mnist.train.images
trainlabels = mnist.train.labels
testimgs = mnist.test.images
testlabels = mnist.test.labels

ntrain,ntest,dim,nclasser = trainimgs.shape[0],testimgs.shape[0],trainimgs.shape[1],testlabels.shape[1]
print(ntrain,ntest,dim,nclasser)
print('MNIST READY')

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
55000 10000 784 10
MNIST READY


In [32]:
# 定义网络
diminput = 28  #  1*28
dimhidden = 128 # 128 个神经元
dimoutput = nclasser # 10个类别
nsteps = 28 #代表将输入拆分成28份

weights = {
    'hidden':tf.Variable(tf.random_normal([diminput,dimhidden])),
    'out':tf.Variable(tf.random_normal([dimhidden,dimoutput]))
}
biases = {
    'hidden':tf.Variable(tf.random_normal([dimhidden])),
    'out':tf.Variable(tf.random_normal([dimoutput]))
}

In [67]:
def RNN(_X,_W,_b,_nsteps,_name):
    # RNN 对输入进的  X进行转换  [batchsize ,nsteps,dimimput]
    # => [nsteps,bathsize,diminput]   转换成5*28个切片 每个切片是28
    _X = tf.transpose(_X,[1,0,2])
    # reshape 将本来的转为   一共做多少个  [nsteps*batchsize,diminput]
    _X = tf.reshape(_X,[-1,diminput])
    # Input layer => hidden layer
    _H = tf.matmul(_X,_W['hidden'])+ _b['hidden']
    # 将所有的整体的隐藏输出进行切片操作
    _Hsplit = tf.split(_H,_nsteps,0)   # 0 代表按行切分  切成nsteps份
    # 开始RNN
    with tf.variable_scope(_name) as scpoe:  # 
        #指向同一个位置 变量共享
        scpoe.reuse_variables()
        lstm_cell =  tf.contrib.rnn.BasicLSTMCell(dimhidden,forget_bias=1.0) # 最终想得到变量的大小  刚开始什么都不忘    
        _LSTM_O,_LSTM_S =tf.contrib.rnn.static_rnn(lstm_cell,_Hsplit,dtype=tf.float32)   # 一个切片一个切片的传入到RNN中训练
    # output  只需要最后的一个输出值
    _O = tf.add(tf.matmul(_LSTM_O[-1],_W['out']),_b['out'])
    # return 
    return {
        'X':_X,
        'H':_H,
        'Hsplit':_Hsplit,
        'LSTM_O':_LSTM_O,
        'LSTM_S':_LSTM_S,
        'O':_O
    }
print('Network ready')

Network ready


In [69]:

learning_rate = 0.001
X = tf.placeholder(tf.float32,[None,nsteps,diminput])
y = tf.placeholder(tf.float32,[None,dimoutput])

myrnn = RNN(X,weights,biases,nsteps,'basic')
pred = myrnn['O']
# 求出损失值
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred,labels=y))
# optm
optm = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
accr = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(pred,1),tf.argmax(y,1)),tf.float32))
init = tf.global_variables_initializer()
print('Network Ready')

Network Ready


In [77]:
train_epoch = 30
batch_size = 16
display_step = 1
with tf.Session() as sess:
    
    sess.run(init)
    for epoch in range(train_epoch):
        #     total_bath = int(mnist.train.num_examples/batch_size)
        avg_cost = 0
        total_batch = 100
        for i in range(total_batch):
            # 每次读16个
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            batch_xs = batch_xs.reshape((batch_size,nsteps,diminput))
            feeds = {X:batch_xs,y:batch_ys}
            sess.run(optm,feed_dict=feeds)
            avg_cost += sess.run(cost, feed_dict=feeds)
        avg_cost = avg_cost/total_batch
        if epoch % display_step == 0: 
            print ("Epoch: %03d/%03d cost: %.9f" % (epoch, train_epoch, avg_cost))
            feeds = {X: batch_xs, y: batch_ys}
            train_acc = sess.run(accr, feed_dict=feeds)
            testimgs = testimgs.reshape((ntest, nsteps, diminput))
            feeds = {X: testimgs[:10], y: testlabels[:10]}
            test_acc = sess.run(accr, feed_dict=feeds)
            print (" Test accuracy: %.3f" % (test_acc))
print ("Optimization Finished.")

Epoch: 000/030 cost: 1.951690043
 Test accuracy: 0.200
Epoch: 001/030 cost: 1.520543435
 Test accuracy: 0.300
Epoch: 002/030 cost: 1.248297115
 Test accuracy: 0.500
Epoch: 003/030 cost: 1.139030192
 Test accuracy: 0.400
Epoch: 004/030 cost: 0.986344287
 Test accuracy: 0.700
Epoch: 005/030 cost: 0.854012781
 Test accuracy: 0.800
Epoch: 006/030 cost: 0.808693137
 Test accuracy: 0.600
Epoch: 007/030 cost: 0.706826313
 Test accuracy: 0.600
Epoch: 008/030 cost: 0.689690714
 Test accuracy: 0.600
Epoch: 009/030 cost: 0.635670507
 Test accuracy: 0.600
Epoch: 010/030 cost: 0.577382978
 Test accuracy: 0.600
Epoch: 011/030 cost: 0.566870750
 Test accuracy: 0.600
Epoch: 012/030 cost: 0.525428522
 Test accuracy: 0.500
Epoch: 013/030 cost: 0.518753212
 Test accuracy: 0.700
Epoch: 014/030 cost: 0.504432921
 Test accuracy: 0.500
Epoch: 015/030 cost: 0.457615062
 Test accuracy: 0.500
Epoch: 016/030 cost: 0.433833105
 Test accuracy: 0.600
Epoch: 017/030 cost: 0.416714282
 Test accuracy: 0.600
Epoch: 018